In [1]:
!nvidia-smi

Wed Dec 22 15:05:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install -U spacy-nightly --pre

In [1]:
# wheel package
!pip install -U pip setuptools wheel

In [2]:
# clone spacy's relation extraction repo
!python -m spacy project clone tutorials/rel_component

✔ Cloned 'tutorials/rel_component' from explosion/projects
/content/rel_component
✔ Your project is now ready!
To fetch the assets, run:
python -m spacy project assets /content/rel_component


In [3]:
# install transformer pipeline and spacy transformer library
!python -m spacy download en_core_web_trf
!pip install -U spacy tranformers

     |████████████████████████████████| 459.7 MB 1.3 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 6.3 MB 9.7 MB/s            
     |████████████████████████████████| 42 kB 25.2 MB/s            
     |████████████████████████████████| 42 kB 29.3 MB/s            
     |████████████████████████████████| 40 kB 49.1 MB/s            
     |████████████████████████████████| 2.6 MB 49.1 MB/s            
     |████████████████████████████████| 1.1 MB 47.5 MB/s            
     |████████████████████████████████| 895 kB 46.7 MB/s            
     |████████████████████████████████| 3.3 MB 54.0 MB/s            
     |████████████████████████████████| 596 kB 48.5 MB/s            
  Created wheel for en-core-web-trf: filename=en_core_web_trf-3.0.0-py3-none-any.whl size=459708991 sha256=457fc581f2d3739320fc3ea080f4eb04574506468fbea26a628054162f76506a
  Stored in directory: /tmp/pip-ephem-wheel-cache-509etuns/wheels/be/d9/92/c65214265163256ecc7588e64

In [4]:
cd rel_component

/content/rel_component


In [ ]:
# create a data folder and upload train,test,dev .spacy file
# open project.yml file and update path of the train,test,and dev
# you can change transformer model by using configs/rel_trf.cfg
# before start training decrease max_length in configs/rel_trf.cfg from default 100 token to 20to increase the effeciency of our model.
# max_length corresponds to max_distance between two entities above which they will not considered for relation classification.
# two entittes from a document will be classified as long as they are within maximum distance of each other.

In [5]:
!spacy project run train_gpu # command to train train transformers
!spacy project run evaluate # command to evaluate on test dataset


================================= train_gpu =================================
Running command: /usr/bin/python3 -m spacy train configs/rel_trf.cfg --output training --paths.train /content/rel_component/data/relations_training.spacy --paths.dev /content/rel_component/data/relations_dev.spacy -c ./scripts/custom_functions.py --gpu-id 0
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2021-12-22 15:13:58,175] [INFO] Set up nlp object from config
[2021-12-22 15:13:58,186] [INFO] Pipeline: ['transformer', 'relation_extractor']
[2021-12-22 15:13:58,192] [INFO] Created vocabulary
[2021-12-22 15:13:58,192] [INFO] Finished initializing nlp object
Downloading: 100% 481/481 [00:00<00:00, 412kB/s]
Downloading: 100% 899k/899k [00:00<00:00, 2.16MB/s]
Downloading: 100% 456k/456k [00:00<00:00, 1.12MB/s]
Downloading: 100% 1.36M/1.36M [00:00<00:00, 2.72MB/s]
Downloading: 100% 501M/501M [00:13<00:00, 36.0MB/s]
Some weights of the model checkpoint at roberta-

### Joint entity and relation extraction pipeline

In [ ]:
# install spacy transformers and transformer pipeline
!pip install -U spacy
!python -m spacy download en_core_web_trf

In [ ]:
# Load the NER model and extract entities:

In [ ]:
nlp = spacy.load("/content/output/model-best")

In [ ]:
text =["""
2+ years of non-internship professional software development experience
Programming experience with at least one modern language such as Java, C++, or C# including object-oriented design.
1+ years of experience contributing to the architecture and design (architecture, design patterns, reliability and scaling) of new and current systems.
Bachelor / MS Degree in Computer Science. Preferably a PhD in data science.
8+ years of professional experience in software development. 2+ years of experience in project management.
Experience in mentoring junior software engineers to improve their skills, and make them more effective, product software engineers.
Experience in data structures, algorithm design, complexity analysis, object-oriented design.
3+ years experience in at least one modern programming language such as Java, Scala, Python, C++, C#
Experience in professional software engineering practices & best practices for the full software development life cycle, including coding standards, code reviews, source control management, build processes, testing, and operations
Experience in communicating with users, other technical teams, and management to collect requirements, describe software product features, and technical designs.
Experience with building complex software systems that have been successfully delivered to customers
Proven ability to take a project from scoping requirements through actual launch of the project, with experience in the subsequent operation of the system in production
"""]

In [ ]:
for doc in nlp.pipe(text, disable= ["tagger"]):
  print(f"spans :{[(e.start,e.text,e.label_) for e in doc.ents]}")

In [ ]:
import spacy
import random
import typer
from pathlib import Path
from spacy.tokens import DocBin, Doc
from spacy.training.example import Example
from rel_pipe import make_relation_extractor, score_relations
from rel_model import create_relation_model, create_classification_layer, create_instances, create_tensors

In [ ]:
nlp2 = spacy.load("/content/rel_component/training/model-best")

In [ ]:
#  we take the entities generated from the NER pipeline and input them to REL pipeline
for name, proc in nlp2.pipeline:
          doc = proc(doc)

In [ ]:
# here we split the paragraph into sentences and apply the relation entraction for each pair of entites found in each sentence.
for value, rel_dict in doc._.rel.items():
  for sent in doc.sents:
    for e in sent.ents:
      for b in sent.ents:
        if e.start == value[0] and b.start == value[1]:
          if rel_dict["EXPERIENCE_IN"] >= 0.9:
            print(f"entities:{e.text, b.text} --> predicted relation :{rel_dict}")
